In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics as stat
import math
import pickle
import gzip
import sklearn
import graphviz
from graphviz import Source
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn import svm

In [44]:
#read in csv files. drop unknown column
traindf = pd.read_csv('train_data.csv', engine='python')
traindf.drop(traindf.columns[traindf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

#drop the id information becuse it does not matter
traindf.drop(traindf.columns[traindf.columns.str.contains('id',case = False)],axis = 1, inplace = True)

#clean the date information
traindf['date'] = traindf['date'].str.replace("(T).*","").astype(int)

traindf.info()
traindf.head()


#Creates a copy of the training df so we can arrange our data and correctly format it
traindfy = traindf.copy()
traindfX = traindf.copy()


#creates y and X df
y = traindfy
X = traindfX

#Shows us our data for X
X.drop(['price'], axis=1, inplace=True)
traindfX.columns = range(traindfX.shape[1])
X.head(10), traindf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17290 entries, 0 to 17289
Data columns (total 20 columns):
date             17290 non-null int32
bedrooms         17290 non-null int64
bathrooms        17290 non-null float64
sqft_living      17290 non-null int64
sqft_lot         17290 non-null int64
floors           17290 non-null float64
waterfront       17290 non-null int64
view             17290 non-null int64
condition        17290 non-null int64
grade            17290 non-null int64
sqft_above       17290 non-null int64
sqft_basement    17290 non-null int64
yr_built         17290 non-null int64
yr_renovated     17290 non-null int64
zipcode          17290 non-null int64
lat              17290 non-null float64
long             17290 non-null float64
sqft_living15    17290 non-null int64
sqft_lot15       17290 non-null int64
price            17290 non-null int64
dtypes: float64(4), int32(1), int64(15)
memory usage: 2.6 MB


(          0  1     2     3      4    5  6  7  8  9    10   11    12  13  \
 0  20141002  3  1.75  2190   7021  1.0  0  2  4  7  1390  800  1953   0   
 1  20150105  3  2.50  2370   6840  2.0  0  0  3  9  2370    0  1987   0   
 2  20140617  2  1.00  1230   3800  1.0  0  0  3  7  1230    0  1928   0   
 3  20141124  4  2.25  2510   9963  1.0  0  0  4  9  2200  310  1967   0   
 4  20140818  3  1.00  1160   7491  1.0  0  0  4  6  1160    0  1917   0   
 5  20150304  4  1.00  1220   7980  1.5  0  0  3  6  1220    0  1944   0   
 6  20141007  4  2.50  1960   5238  2.0  0  0  3  7  1960    0  2003   0   
 7  20140819  3  2.50  2060   4812  2.0  0  0  3  8  2060    0  1990   0   
 8  20150407  4  2.50  2700  17853  2.0  0  0  4  9  2700    0  1960   0   
 9  20140721  4  2.25  2470  17008  2.0  0  0  4  8  2470    0  1979   0   
 
       14       15       16    17     18  
 0  98178  47.5033 -122.232  2180   7155  
 1  98119  47.6503 -122.366  1590   4400  
 2  98115  47.6797 -122.292  1610

In [45]:
#shows us our data for y 
y = traindfy['price']
y.head()

0    353000
1    300523
2    435000
3    800000
4    417500
Name: price, dtype: int64

In [46]:
#split the data (necessary for other classifiers)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
#checking for correct array sizes
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12967, 19)
(4323, 19)
(12967,)
(4323,)


In [48]:
#actual linear regression
Lregression = LinearRegression().fit(X.values, y)
print("LinReg Score: ",Lregression.score(X, y))


LinReg Score:  0.70519564705033


In [ ]:
#Log Regress don't run it takes too long
#Logreg = LogisticRegression(solver='liblinear', random_state=0).fit(X, y)
#print("LogReg Score: ",Logreg.score(X,y))

In [7]:
#***dont run takes foreer will look at it later*** 
#clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
#clf.score(X_test, y_test)

The decision tree seen below is extremely hard to grade since we are looking at the individual house prices.

The Gini score given is always going to be really high, becuase in all liklihood there is only one house per price.

So instead we need to split the price into ranges, or bins, and then re-enter the correct ranges for the prices.

In [10]:
#decision tree classifier
Dectree = DecisionTreeClassifier(max_depth = 5).fit(X,y)
#Source(export_graphviz(Dectree, out_file = None, feature_names = list(X.columns)))

In [49]:
#Checking the range of the price values
Pricemin = y.min()
Pricemax = y.max()

print("Min is: ", Pricemin, "Max is: ", Pricemax)

#Creating our range
Priceranges = np.linspace(Pricemin,Pricemax,15)

Range_y = y.copy()
for i in range(len(Range_y)):
    for j in range(len(Priceranges)):
        if Range_y[i] <= Priceranges[j+1]: 
            Range_y[i] = str(Priceranges[j])+"-"+str(Priceranges[j+1])
            break
            
Range_y.head()

#Now that we have our dataset lets create a new tree and compare!
Dectree = DecisionTreeClassifier(max_depth = 10).fit(X,Range_y)
Source(export_graphviz(Dectree, out_file = None, feature_names = list(X.columns)))

#Much Better Right!

Min is:  75000 Max is:  5570000
